# im sorry we try again

## pip3 requirements magic

In [3]:
!pip3 install -r ../../requirements.txt
!pip3 install pandas
!pip3 install numpy --upgrade

  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


## import statement magic

In [4]:
# import tensorflow/keras stuff
# from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D, Activation, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import keras
from keras.preprocessing.image import img_to_array

# import data processing/visualisation things
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt
%matplotlib inline

# importing misc stuff
import os
import cv2

## Create the EfficientNetB0 Model

We have decided to do EfficientNetB0, instead of EfficientNetB7, as we originally wanted, due to limited resources. EfficientNetB0 requires inputs of 224 x 224, but B7 requires 600 x 600, placing a significantly higher demand on our resources and causing strain for us.

From Keras:
>This model takes input images of shape (224, 224, 3), and the input data should be in the range [0, 255]. Normalization is included as part of the model.

In [5]:
from tensorflow.keras.applications import EfficientNetB0

# include_top = False as we want to do transfer learning
base_model = EfficientNetB0(include_top=False, weights='imagenet')

## Load the data in

### set up the data generator, and have it rescale pixel values to 0, 1

In [6]:
datagen = ImageDataGenerator(rescale=1./255)  # rescale pixel values to [0,1]

### set up some default values (paths + constants)

In [7]:
# IMG_SIZE is determined by EfficientNet model choice
directory = 'lung_colon_image_set/lung_image_sets/'
batch_size = 32
IMG_SIZE = 224  # adjust according to your image size
NUM_CLASSES = 3 # we have three classes (healthy, adenocarcinoma, and SCC)
TARGET_SIZE = (IMG_SIZE, IMG_SIZE)

### set up the data generator

In [8]:
data_generator = datagen.flow_from_directory(
    directory,
    target_size = TARGET_SIZE, # image size of 224 x 224
    batch_size=batch_size,
    class_mode='categorical'  # we're dealing with categorical data, this also does one-hot encoding for us
)

Found 15000 images belonging to 3 classes.


### Create the model

In [9]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

## detour: attempting to load the data in as a pandas df to save myself

In [10]:
# ignore this didn't work

## attempt

## TRAINING START

In [ ]:
# Split data into five folds for validation
X = np.array(data_generator.filenames)
kf = KFold(n_splits=5, shuffle=True)

for train_index, val_index in kf.split(X):
    # get the train and test values for this k fold
    train_files, val_files = X[train_index], X[val_index]

    train_images = [keras.utils.load_img(directory + file) for file in train_files]

    train_array = [keras.utils.img_to_array(img) for img in train_images]
    
    val_images = [keras.utils.load_img(directory + file) for file in val_files]
    
    train_array = [keras.utils.img_to_array(img) for img in val_images]

    # build the model!
    model = build_model(num_classes=NUM_CLASSES)

    # train!!!
    epochs = 8  # @param {type: "slider", min:8, max:80}
    hist = model.fit(train_array, epochs=epochs, validation_data=val_array)

    # plot
    plot_hist(hist)